# 6장 .  한글 챗봇 (데이터, seq2seq, 트랜스포머)



# =========================




# =========================


# 6.3 : 시퀀스 투 시퀀스 모델링



# 한글 챗봇 파이썬 코드 (seq2seq 모델)

## data_outseq2 디렉토리 사용

# 패키지 불러오기

In [1]:
# import tensorflow as tf
# 텐서플로우 2.0 사용 


import tensorflow.compat.v1  as tf


# import model as ml == 자체 사용 
# import data == 자체 시용 

import numpy as np
import os
import sys



#from configs import DEFINES == 자체 사용 
#########################################


print( tf.__version__ )

2.0.0


In [2]:
#-*- coding: utf-8 -*-
#import tensorflow as tf
#import tensorflow.compat.v1  as tf


tf.app.flags.DEFINE_string('f', '', 'kernel') 
# 주피터에서 커널에 전달하기 위한 프레그 방법


tf.app.flags.DEFINE_integer('batch_size', 64, 'batch size') # 배치 크기
tf.app.flags.DEFINE_integer('train_steps', 25000, 'train steps') # 학습 에포크
tf.app.flags.DEFINE_float('dropout_width', 0.5, 'dropout width') # 드롭아웃 크기
tf.app.flags.DEFINE_integer('layer_size', 3, 'layer size') # 멀티 레이어 크기 (multi rnn)
tf.app.flags.DEFINE_integer('hidden_size', 128, 'weights size') # 가중치 크기
tf.app.flags.DEFINE_float('learning_rate', 1e-3, 'learning rate') # 학습률
tf.app.flags.DEFINE_string('data_path', './data_in/ChatBotData.csv', 'data path') #  데이터 위치
tf.app.flags.DEFINE_string('vocabulary_path', './data_outseq2/vocabularyData.voc', 'vocabulary path') # 사전 위치
tf.app.flags.DEFINE_string('check_point_path', './data_outseq2/check_point', 'check point path') # 체크 포인트 위치
tf.app.flags.DEFINE_integer('shuffle_seek', 1000, 'shuffle random seek') # 셔플 시드값
tf.app.flags.DEFINE_integer('max_sequence_length', 25, 'max sequence length') # 시퀀스 길이
tf.app.flags.DEFINE_integer('embedding_size', 128, 'embedding size') # 임베딩 크기
tf.app.flags.DEFINE_boolean('tokenize_as_morph', True, 'set morph tokenize') # 형태소에 따른 토크나이징 사용 유무
tf.app.flags.DEFINE_boolean('embedding', True, 'Use Embedding flag') # 임베딩 유무 설정
tf.app.flags.DEFINE_boolean('multilayer', True, 'Use Multi RNN Cell') # 멀티 RNN 유무


# Define FLAGS


DEFINES = tf.app.flags.FLAGS

#############################

# import tensorflow.compat.v1  as tf  에서만  tf.app

# 여러번 실행시키면 에러 
# kernel 을 restart 시킴 

In [3]:
# tf.app.flags.DEFINE_string('f', '', 'kernel')
# 어떤 경우에는 필요 f 가 필요한다고 하는 경우
# ./../ 란 이전 디렉토리를 말함 
# ./../data_in/ChatBotData.csv

print( DEFINES.data_path )

./data_in/ChatBotData.csv


# 챗봇 데이터와 단어장 연습

In [8]:
import pandas as pd



data = pd.read_csv( DEFINES.data_path , encoding='utf-8')

data.head() 

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [15]:
print( data['Q'][0:5])

0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object


In [14]:
print( data['label'][0:5])

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64


In [30]:
from konlpy.tag import Okt # Twitter


from tqdm import tqdm



questions, answers  = list( data['Q'][0:5] ) , list( data['A'][0:5] )


#if DEFINES.tokenize_as_morph:  # 형태소에 따른 토크나이져 처리
#questions = prepro_like_morphlized( questions )
###########################################
#answers = prepro_like_morphlized( answers )

    
    
morph_analyzer = Okt()  # Twitter()
# 형태소 토크나이즈 결과 문장을 받을
#  리스트를 생성합니다.

result_data = list()
# 데이터에 있는 매 문장에 대해 토크나이즈를
# 할 수 있도록 반복문을 선언합니다.


# Twitter.morphs 함수를 통해 토크나이즈 된
# 리스트 객체를 받고 다시 공백문자를 기준으로
# 하여 문자열로 재구성 해줍니다.
for seq in tqdm( questions + answers ):
    morphlized_seq = " ".join(morph_analyzer.morphs(seq.replace(' ', '')))
    result_data.append(morphlized_seq)
    
    
questions = result_data         
        
        
datas = []
# 질문과 답변을 extend을 통해서 구조가 없는 배열로 만든다.
datas.extend(questions)
# datas.extend(answers)

print( datas )



100%|█████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 911.45it/s]

['12시 땡 !', '1 지망 학교 떨어졌어', '3 박 4일 놀러가고싶다', '3 박 4일 정도놀러가고싶다', 'PPL 심하네', '하루 가 또 가네요 .', '위로 해드립니다 .', '여행 은 언제나 좋죠 .', '여행 은 언제나 좋죠 .', '눈살 이 찌푸려지죠 .']


['12시 땡 !', '1 지망 학교 떨어졌어', '3 박 4일 놀러가고싶다', '3 박 4일 정도놀러가고싶다', 'PPL 심하네', '하루 가 또 가네요 .', '위로 해드립니다 .', '여행 은 언제나 좋죠 .', '여행 은 언제나 좋죠 .', '눈살 이 찌푸려지죠 .']

In [31]:

import re

FILTERS = "([~.,!?\"':;)(])"
CHANGE_FILTER = re.compile(FILTERS)


vocabwords = []
for sentence in datas:
    # FILTERS = "([~.,!?\"':;)(])"
    # 위 필터와 같은 값들을 정규화 표현식을 
    # 통해서 모두 "" 으로 변환 해주는 부분이다.
    sentence = re.sub(CHANGE_FILTER, "", sentence)
    for word in sentence.split():
        vocabwords.append(word)
            
print( vocabwords )

['12시', '땡', '1', '지망', '학교', '떨어졌어', '3', '박', '4일', '놀러가고싶다', '3', '박', '4일', '정도놀러가고싶다', 'PPL', '심하네', '하루', '가', '또', '가네요', '위로', '해드립니다', '여행', '은', '언제나', '좋죠', '여행', '은', '언제나', '좋죠', '눈살', '이', '찌푸려지죠']


['12시', '땡', '1', '지망', '학교', '떨어졌어', '3', '박', '4일', '놀러가고싶다', '3', '박', '4일', '정도놀러가고싶다', 'PPL', '심하네', '하루', '가', '또', '가네요', '위로', '해드립니다', '여행', '은', '언제나', '좋죠', '여행', '은', '언제나', '좋죠', '눈살', '이', '찌푸려지죠']

# 챗봇 데이터 

In [18]:
# from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split




def load_data():
    # 판다스를 통해서 데이터를 불러온다.
    data_df = pd.read_csv(DEFINES.data_path, header=0)
    # 질문과 답변 열을 가져와 question과 answer에 넣는다.
    question, answer = list(data_df['Q']), list(data_df['A'])
    # skleran에서 지원하는 함수를 통해서 학습 셋과 
    # 테스트 셋을 나눈다.
    
    train_input, eval_input, train_label, eval_label = train_test_split(question, answer, test_size=0.33, random_state=42)
    
    # 그 값을 리턴한다.
    return train_input, train_label, eval_input, eval_label




train_input, train_label, eval_input, eval_label = load_data()



In [19]:
print( train_input[0:5] )

['짝사랑 했던 여자가 떠나갔네.', '대리 불렀는데 안 오네', '내가 싫은 건 너도 싫어해줘서 고마워', '썸은 왜 타?', '짝녀 프사 자음이 궁금함.']


In [20]:
print( train_label[0:5] )

['여기까지 인연이었나봅니다.', '잘했어요.', '싫은 게 통해야 편하죠.', '사귀기 전에 마음을 확인하는 단계라서 그렇지 않을까요?', '궁금할 수 있어요.']


## 사전 만들기

In [4]:
# data.py 파일 
#import tensorflow as tf
#import os
#from configs import DEFINES

import enum

import numpy as np

from konlpy.tag import Okt # Twitter

from tqdm import tqdm


PAD_MASK = 0
NON_PAD_MASK = 1

FILTERS = "([~.,!?\"':;)(])"
PAD = "<PADDING>"
STD = "<START>"
END = "<END>"
UNK = "<UNKNWON>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]
CHANGE_FILTER = re.compile(FILTERS)




def load_vocabulary():
    # 사전을 담을 배열 준비한다.
    vocabulary_list = []
    # 사전을 구성한 후 파일로 저장 진행한다. 
    # 그 파일의 존재 유무를 확인한다.
    if (not (os.path.exists(DEFINES.vocabulary_path))):
        # 이미 생성된 사전 파일이 존재하지 않으므로 
        # 데이터를 가지고 만들어야 한다.
        # 그래서 데이터가 존재 하면 사전을 만들기 위해서 
        # 데이터 파일의 존재 유무를 확인한다.
        if (os.path.exists(DEFINES.data_path)):
            # 데이터가 존재하니 판단스를 통해서 
            # 데이터를 불러오자
            data_df = pd.read_csv(DEFINES.data_path, encoding='utf-8')
            # 판다스의 데이터 프레임을 통해서 
            # 질문과 답에 대한 열을 가져 온다.
            question, answer = list(data_df['Q']), list(data_df['A'])
            if DEFINES.tokenize_as_morph:  # 형태소에 따른 토크나이져 처리
                question = prepro_like_morphlized(question)
                ###########################################
                answer = prepro_like_morphlized(answer)
                
            data = []
            # 질문과 답변을 extend을 
            # 통해서 구조가 없는 배열로 만든다.
            data.extend(question)
            data.extend(answer)
            # 토큰나이져 처리 하는 부분이다.
            
            words = data_tokenizer(data)
            ############################
            
            
            # 공통적인 단어에 대해서는 모두 
            # 필요 없으므로 한개로 만들어 주기 위해서
            # set해주고 이것들을 리스트로 만들어 준다.
            words = list(set(words))
            # 데이터 없는 내용중에 MARKER를 사전에 
            # 추가 하기 위해서 아래와 같이 처리 한다.
            # 아래는 MARKER 값이며 리스트의 첫번째 부터 
            # 순서대로 넣기 위해서 인덱스 0에 추가한다.
            # PAD = "<PADDING>"
            # STD = "<START>"
            # END = "<END>"
            # UNK = "<UNKNWON>"     
            words[:0] = MARKER
        # 사전을 리스트로 만들었으니 이 내용을 
        # 사전 파일을 만들어 넣는다.
        with open(DEFINES.vocabulary_path, 'w') as vocabulary_file:
            for word in words:
                vocabulary_file.write(word + '\n')

    # 사전 파일이 존재하면 여기에서 
    # 그 파일을 불러서 배열에 넣어 준다.
    with open(DEFINES.vocabulary_path, 'r', encoding='utf-8') as vocabulary_file:
        for line in vocabulary_file:
            vocabulary_list.append(line.strip())

    # 배열에 내용을 키와 값이 있는 
    # 딕셔너리 구조로 만든다.
    char2idx, idx2char = make_vocabulary(vocabulary_list)
    ######################################################
    
    # 두가지 형태의 키와 값이 있는 형태를 리턴한다. 
    # (예) 단어: 인덱스 , 인덱스: 단어)
    return char2idx, idx2char, len(char2idx)




def make_vocabulary(vocabulary_list):
    # 리스트를 키가 단어이고 값이 인덱스인 
    # 딕셔너리를 만든다.
    char2idx = {char: idx for idx, char in enumerate(vocabulary_list)}
    # 리스트를 키가 인덱스이고 값이 단어인 
    # 딕셔너리를 만든다.
    idx2char = {idx: char for idx, char in enumerate(vocabulary_list)}
    # 두개의 딕셔너리를 넘겨 준다.
    return char2idx, idx2char




def prepro_like_morphlized(data):
    # 형태소 분석 모듈 객체를
    # 생성합니다.

    morph_analyzer = Okt()  # Twitter()
    # 형태소 토크나이즈 결과 문장을 받을
    #  리스트를 생성합니다.
    result_data = list()
    # 데이터에 있는 매 문장에 대해 토크나이즈를
    # 할 수 있도록 반복문을 선언합니다.
    for seq in tqdm(data):
        # Twitter.morphs 함수를 통해 토크나이즈 된
        # 리스트 객체를 받고 다시 공백문자를 기준으로
        # 하여 문자열로 재구성 해줍니다.
        morphlized_seq = " ".join(morph_analyzer.morphs(seq.replace(' ', '')))
        result_data.append(morphlized_seq)

    return result_data




def data_tokenizer(data):
    # 토크나이징 해서 담을 배열 생성
    words = []
    for sentence in data:
        # FILTERS = "([~.,!?\"':;)(])"
        # 위 필터와 같은 값들을 정규화 표현식을 
        # 통해서 모두 "" 으로 변환 해주는 부분이다.
        sentence = re.sub(CHANGE_FILTER, "", sentence)
        for word in sentence.split():
            words.append(word)
    # 토그나이징과 정규표현식을 통해 만들어진 
    # 값들을 넘겨 준다.
    return [word for word in words if word]







In [5]:
DATA_OUT_PATH = './data_outseq2/'  # 현재 디렉토리에서의 서브디렉토리

data_out_path = os.path.join(os.getcwd(), DATA_OUT_PATH)

os.makedirs( data_out_path, exist_ok=True )


char2idx,  idx2char, vocabulary_length = load_vocabulary()




###################################################################


## 함수

    def enc_processing(value, dictionary):
    
    def dec_input_processing(value, dictionary):
    
    def dec_target_processing(value, dictionary):
    
    

In [8]:


# 인덱스화 할 value와 키가 워드이고 
# 값이 인덱스인 딕셔너리를 받는다.
def enc_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는 
    # 배열이다.(누적된다.)
    sequences_input_index = []
    # 하나의 인코딩 되는 문장의 
    # 길이를 가지고 있다.(누적된다.)
    sequences_length = []
    # 형태소 토크나이징 사용 유무
    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)

    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는 
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 하나의 문장을 인코딩 할때 
        # 가지고 있기 위한 배열이다.
        sequence_index = []
        # 문장을 스페이스 단위로 
        # 자르고 있다.
        for word in sequence.split():
            # 잘려진 단어들이 딕셔너리에 존재 하는지 보고 
            # 그 값을 가져와 sequence_index에 추가한다.
            if dictionary.get(word) is not None:
                sequence_index.extend([dictionary[word]])
            # 잘려진 단어가 딕셔너리에 존재 하지 않는 
            # 경우 이므로 UNK(2)를 넣어 준다.
            else:
                sequence_index.extend([dictionary[UNK]])
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length]
        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))
        # max_sequence_length보다 문장 길이가 
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        # 뒤로 넣어 준다. 
        sequence_index.reverse()
        # 인덱스화 되어 있는 값을 
        # sequences_input_index에 넣어 준다.
        sequences_input_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다. 
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 
    # 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 
    # 그 길이를 넘겨준다.  
    return np.asarray(sequences_input_index), sequences_length



###############################################################

# 디코딩 입력 데이터를 만드는 함수이다.
def dec_input_processing(value, dictionary):
    sequences_output_index = []
    sequences_length = []

    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)

    for sequence in value:
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        sequence_index = []
        # 디코딩 입력의 처음에는 START가 와야 하므로 
        # 그 값을 넣어 주고 시작한다.
        sequence_index = [dictionary[STD]] + [dictionary[word] for word in sequence.split()]

        if len(sequence_index) > DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length]
        sequences_length.append(len(sequence_index))
        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        sequences_output_index.append(sequence_index)

    return np.asarray(sequences_output_index), sequences_length

#######################################################################



# 인덱스화 할 value와 키가 워드 이고
# 값이 인덱스인 딕셔너리를 받는다.


# 디코딩 출력 데이터를 만드는 함수이다.
def dec_target_processing(value, dictionary):
    sequences_target_index = []

    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)
    for sequence in value:
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 문장에서 스페이스 단위별로 단어를 가져와서 
        # 딕셔너리의 값인 인덱스를 넣어 준다.
        # 디코딩 출력의 마지막에 END를 넣어 준다.
        sequence_index = [dictionary[word] for word in sequence.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        # 그리고 END 토큰을 넣어 준다
        if len(sequence_index) >= DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length-1] + [dictionary[END]]
        else:
            sequence_index += [dictionary[END]]
        # max_sequence_length보다 문장 길이가 

        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        sequences_target_index.append(sequence_index)

    return np.asarray(sequences_target_index)


#########################################################



def dec_target_processing_two(value, dictionary):
    # 인덱스 값들을 가지고 있는 
    # 배열이다.(누적된다)
    sequences_target_index = []
    sequences_length = []
    # 형태소 토크나이징 사용 유무
    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)
    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는 
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 문장에서 스페이스 단위별로 단어를 가져와서 
        # 딕셔너리의 값인 인덱스를 넣어 준다.
        # 디코딩 출력의 마지막에 END를 넣어 준다.
        sequence_index = [dictionary[word] for word in sequence.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        # 그리고 END 토큰을 넣어 준다
        if len(sequence_index) >= DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length-1] + [dictionary[END]]
        else:
            sequence_index += [dictionary[END]]
        
        # 학습시 PAD 마스크를 위한 벡터를 구성한다.                   
        sequences_length.append([PAD_MASK if num > len(sequence_index) else NON_PAD_MASK for num in range (DEFINES.max_sequence_length)])
        # max_sequence_length보다 문장 길이가 
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을 
        # sequences_target_index에 넣어 준다.
        sequences_target_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다. 
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_target_index), np.asarray(sequences_length)

##############################################################



In [9]:


#TODO3: 실행 안됨, 실행 후 지속적으로 리뷰하겠음

# 훈련셋 인코딩 만드는 부분이다.
train_input_enc, train_input_enc_length = enc_processing(train_input, char2idx)

# 훈련셋 디코딩 입력 부분 만드는 부분이다.
train_output_dec, train_output_dec_length = dec_input_processing(train_label, char2idx) 
#TODO1 실행 안되어 확인 필요(AttributeError: module 'data' has no attribute 'dec_output_processing)

# 훈련셋 디코딩 출력 부분 만드는 부분이다.
train_target_dec = dec_target_processing(train_label, char2idx)



############################################################


# 평가셋 인코딩 만드는 부분이다.
eval_input_enc, eval_input_enc_length = enc_processing(eval_input,char2idx)

# 평가셋 인코딩 만드는 부분이다.
eval_output_dec, eval_output_dec_length = dec_input_processing(eval_label, char2idx)

# 평가셋 인코딩 만드는 부분이다.
eval_target_dec = dec_target_processing(eval_label, char2idx)



100%|█████████████████████████████████████████████████████████████████████████| 3902/3902 [00:09<00:00, 393.08it/s]


# 체크 포인트 경로 만들기

In [10]:
# 현재 경로'./'에 현재 경로 하부에 
# 체크 포인트를 저장한 디렉토리를 설정한다.



check_point_path = os.path.join(os.getcwd(), DEFINES.check_point_path)




# 디렉토리를 만드는 함수이며 두번째 인자 exist_ok가 
# True이면 디렉토리가 이미 존재해도 OSError가 
# 발생하지 않는다.
# exist_ok가 False이면 이미 존재하면 
# OSError가 발생한다.


os.makedirs(check_point_path, exist_ok=True)

# LSTM 네트워크와 에스티메이터 모델 구성

# 클래시파이어  classifier = tf.estimator.Estimator(

In [11]:
# version 2 22222222222222222222222222

#-*- coding: utf-8 -*-
#import tensorflow as tf
#import sys
#from configs import DEFINES




# 엘에스티엠(LSTM) 단층 네트워크 구성하는 부분
def make_lstm_cell(mode, hiddenSize, index):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hiddenSize, name = "lstm"+str(index))
    if mode == tf.estimator.ModeKeys.TRAIN:
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=DEFINES.dropout_width)
        # tensor 2.0  tf.contrib.rnn.
        #############################
    return cell





# 에스티메이터 모델 부분이다.
def mlmodel(features, labels, mode, params):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    
    # 인코딩 부분 (미리 정의된 임베딩 벡터 사용 유무)
    if params['embedding'] == True:
        # 가중치 행렬에 대한 초기화 함수이다.
        # xavier (Xavier Glorot와 Yoshua Bengio (2010)
        # URL : http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf
        #initializer = tf.contrib.layers.xavier_initializer()
        initializer = tf.glorot_uniform_initializer()
        
        embedding = tf.get_variable(name = "embedding", # 이름
                                 	  shape=[params['vocabulary_length'], params['embedding_size']], #  모양
                                 	  dtype=tf.float32, # 타입
                                 	  initializer=initializer, # 초기화 값
                                 	  trainable=True) # 학습 유무
    else:   
        # tf.eye를 통해서 사전의 크기 만큼의 단위행렬 
        # 구조를 만든다.
        embedding = tf.eye(num_rows = params['vocabulary_length'], dtype = tf.float32)
        embedding = tf.get_variable(name = "embedding", # 이름
                                            initializer = embedding, # 초기화 값
                                            trainable = False) # 학습 유무

    # 임베딩된 인코딩 배치를 만든다.
    embedding_encoder = tf.nn.embedding_lookup(params = embedding, ids = features['input'])

    # 임베딩된 디코딩 배치를 만든다.
    embedding_decoder = tf.nn.embedding_lookup(params = embedding, ids = features['output'])

    with tf.variable_scope('encoder_scope', reuse=tf.AUTO_REUSE):
        # 값이 True이면 멀티레이어로 모델을 구성하고 False이면 
        # 단일레이어로 모델을 구성 한다.
        if params['multilayer'] == True:
            encoder_cell_list = [make_lstm_cell(mode, params['hidden_size'], i) for i in range(params['layer_size'])]
            #rnn_cell = tf.contrib.rnn.MultiRNNCell(encoder_cell_list)
            rnn_cell = tf.nn.rnn_cell.MultiRNNCell(encoder_cell_list)
        else:
            rnn_cell = make_lstm_cell(mode, params['hidden_size'], "")
        
        # rnn_cell에 의해 지정된 dynamic_rnn 반복적인 신경망을 만든다. 
        # encoder_states 최종 상태  [batch_size, cell.state_size]
        encoder_outputs, encoder_states = tf.nn.dynamic_rnn(cell=rnn_cell, # RNN 셀
                                                                inputs=embedding_encoder, # 입력 값
                                                                dtype=tf.float32) # 타입

    with tf.variable_scope('decoder_scope', reuse=tf.AUTO_REUSE):
        if params['multilayer'] == True:
            decoder_cell_list = [make_lstm_cell(mode, params['hidden_size'], i) for i in range(params['layer_size'])]
            #rnn_cell = tf.contrib.rnn.MultiRNNCell(decoder_cell_list)
            rnn_cell = tf.nn.rnn_cell.MultiRNNCell(encoder_cell_list)
        else:
            rnn_cell = make_lstm_cell(mode, params['hidden_size'], "")

        decoder_initial_state = encoder_states
        decoder_outputs, decoder_states = tf.nn.dynamic_rnn(cell=rnn_cell, # RNN 셀
                       inputs=embedding_decoder, # 입력 값
                       initial_state=decoder_initial_state, # 인코딩의 마지막 값으로 초기화
                       dtype=tf.float32) # 타입


    # logits는 마지막 히든레이어를 통과한 결과값이다.
    logits = tf.layers.dense(decoder_outputs, params['vocabulary_length'], activation=None)

	# argmax를 통해서 최대 값을 가져 온다.
    predict = tf.argmax(logits, 2)

    if PREDICT:
        predictions = { # 예측 값들이 여기에 딕셔너리 형태로 담긴다.
            'indexs': predict, # 시퀀스 마다 예측한 값
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    #  
    # logits과 같은 차원을 만들어 마지막 결과 값과 정답 값을 비교하여 에러를 구한다.
    labels_ = tf.one_hot(labels, params['vocabulary_length'])
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels_))
    # 라벨과 결과가 일치하는지 빈도 계산을 통해 정확도를 측정하는 방법이다.
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predict,name='accOp')

    # accuracy를 전체 값으로 나눠 확률 값으로 한다.
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])
    
    if EVAL:
        # 에러 값(loss)과 정확도 값(eval_metric_ops) 전달
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

    # 수행 mode(tf.estimator.ModeKeys.TRAIN)가 
    # 아닌 경우는 여기 까지 오면 안되도록 방어적 코드를 넣은것이다.
    assert TRAIN

    optimizer = tf.train.AdamOptimizer(learning_rate=DEFINES.learning_rate)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())  

    # 에러 값(loss)과 그라디언트 반환값 (train_op) 전달
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [12]:
# 에스티메이터 구성한다.

# TODO2: 왜 분류모델이라고 적혀있나요???? s2s이 아닌가요?

classifier = tf.estimator.Estimator(
    
        model_fn = mlmodel, # 모델 등록한다.
    
        model_dir=DEFINES.check_point_path, # 체크포인트 위치 등록한다.
    
        params={ # 모델 쪽으로 파라메터 전달한다.
            'hidden_size': DEFINES.hidden_size, # 가중치 크기 설정한다.
            'layer_size': DEFINES.layer_size, # 멀티 레이어 층 개수를 설정한다.
            'learning_rate': DEFINES.learning_rate, # 학습율 설정한다. 
            'vocabulary_length': vocabulary_length, # 딕셔너리 크기를 설정한다.
            'embedding_size': DEFINES.embedding_size, # 임베딩 크기를 설정한다.
            'embedding': DEFINES.embedding, # 임베딩 사용 유무를 설정한다.
            'multilayer': DEFINES.multilayer, # 멀티 레이어 사용 유무를 설정한다.
            
        })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_outseq2/check_point', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021E72EC81D0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# 클래시파이어 만들기와 학습 실행

In [13]:


def rearrange(input, output, target):
    features = {"input": input, "output": output}
    return features, target



# 학습에 들어가 배치 데이터를 만드는 함수이다.
def train_input_fn(train_input_enc, train_target_dec_length, train_target_dec, batch_size):
    # Dataset을 생성하는 부분으로써 from_tensor_slices부분은 
    # 각각 한 문장으로 자른다고 보면 된다.
    # train_input_enc, train_target_dec_length, train_target_dec 
    # 3개를 각각 한문장으로 나눈다.
    dataset = tf.data.Dataset.from_tensor_slices((train_input_enc, train_target_dec_length, train_target_dec))
    # 전체 데이터를 썩는다.
    dataset = dataset.shuffle(buffer_size=len(train_input_enc))
    # 배치 인자 값이 없다면  에러를 발생 시킨다.
    assert batch_size is not None, "train batchSize must not be None"
    # from_tensor_slices를 통해 나눈것을 
    # 배치크기 만큼 묶어 준다.
    dataset = dataset.batch(batch_size)
    # 데이터 각 요소에 대해서 train_rearrange 함수를 
    # 통해서 요소를 변환하여 맵으로 구성한다.
    
    dataset = dataset.map(rearrange)
    #######################################
    # train_rearrange 에서 그냥 rearrange 로 
    
    # repeat()함수에 원하는 에포크 수를 넣을수 있으면 
    # 아무 인자도 없다면 무한으로 이터레이터 된다.
    dataset = dataset.repeat()
    # make_one_shot_iterator를 통해 이터레이터를 
    # 만들어 준다.
    iterator = dataset.make_one_shot_iterator()
    # 이터레이터를 통해 다음 항목의 텐서 
    # 개체를 넘겨준다.
    return iterator.get_next()


# 평가에 들어가 배치 데이터를 만드는 함수이다.
def dataeval_input_fn(eval_input_enc, eval_target_dec, batch_size):
    # Dataset을 생성하는 부분으로써 from_tensor_slices부분은 
    # 각각 한 문장으로 자른다고 보면 된다.
    # eval_input_enc, eval_target_dec, batch_size 
    # 3개를 각각 한문장으로 나눈다.
    dataset = tf.data.Dataset.from_tensor_slices((eval_input_enc, eval_target_dec))
    # 전체 데이터를 섞는다.
    dataset = dataset.shuffle(buffer_size=len(eval_input_enc))
    # 배치 인자 값이 없다면  에러를 발생 시킨다.
    assert batch_size is not None, "eval batchSize must not be None"
    # from_tensor_slices를 통해 나눈것을 
    # 배치크기 만큼 묶어 준다.
    dataset = dataset.batch(batch_size)
    # 데이터 각 요소에 대해서 rearrange 함수를 
    # 통해서 요소를 변환하여 맵으로 구성한다.
    dataset = dataset.map(rearrange)
    #################################
    
    # repeat()함수에 원하는 에포크 수를 넣을수 있으면 
    # 아무 인자도 없다면 무한으로 이터레이터 된다.
    # 평가이므로 1회만 동작 시킨다.
    dataset = dataset.repeat(1)
    # make_one_shot_iterator를 통해 
    # 이터레이터를 만들어 준다.
    iterator = dataset.make_one_shot_iterator()
    # 이터레이터를 통해 다음 항목의 
    # 텐서 개체를 넘겨준다.
    return iterator.get_next()



#######################################################


In [14]:
# 학습 실행
################################################

# tf.app.flags.DEFINE_integer('train_steps', 25000, 'train steps') # 학습 에포크


classifier.train(input_fn=lambda:  train_input_fn(
    train_input_enc, train_output_dec, train_target_dec,  DEFINES.batch_size), steps=DEFINES.train_steps)




Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer insta

KeyboardInterrupt: 

## 이미 학습된 곳에서 다시 시작하는 모습


    INFO:tensorflow:Calling model_fn.
    INFO:tensorflow:Done calling model_fn.
    INFO:tensorflow:Create CheckpointSaverHook.
    INFO:tensorflow:Graph was finalized.
    INFO:tensorflow:Restoring parameters from ./data_outseq2/check_point\model.ckpt-3615
    
    WARNING:tensorflow:From C:\WinPython37F\python-3.7.2.amd64\lib\site-packages\tensorflow_core\python\training\saver.py:1069: get_checkpoint_mtimes (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
    Instructions for updating:
    Use standard file utilities to get mtimes.
    INFO:tensorflow:Running local_init_op.
    INFO:tensorflow:Done running local_init_op.
    INFO:tensorflow:Saving checkpoints for 3615 into ./data_outseq2/check_point\model.ckpt.
    INFO:tensorflow:loss = 6.9968653, step = 3615
    INFO:tensorflow:global_step/sec: 0.578642
    INFO:tensorflow:loss = 1.5761786, step = 3715 (172.833 sec)
    INFO:tensorflow:global_step/sec: 0.586996
    INFO:tensorflow:loss = 1.469312, step = 3815 (170.349 sec)
    
    ...
    ...
    
    INFO:tensorflow:loss = 0.47262642, step = 22804 (139.072 sec)
    INFO:tensorflow:Saving checkpoints for 22880 into ./data_outseq2/check_point\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.69807
    INFO:tensorflow:loss = 0.51194936, step = 22904 (143.241 sec)
    INFO:tensorflow:global_step/sec: 0.670207
    INFO:tensorflow:loss = 0.46665937, step = 23004 (149.207 sec)
    INFO:tensorflow:global_step/sec: 0.682238
    INFO:tensorflow:loss = 0.5301347, step = 23104 (146.574 sec)
    INFO:tensorflow:loss = 0.4891078, step = 23204 (141.596 sec)
    INFO:tensorflow:Saving checkpoints for 23296 into ./data_outseq2/check_point\model.ckpt.

# 평가 함수

In [15]:

def rearrange(input, output, target):
    features = {"input": input, "output": output}
    return features, target




# 평가에 들어가 배치 데이터를 만드는 함수이다.
def eval_input_fn(eval_input_enc, eval_output_dec, eval_target_dec, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((eval_input_enc, eval_output_dec, eval_target_dec))
    # 전체 데이터를 섞는다.
    dataset = dataset.shuffle(buffer_size=len(eval_input_enc))
    assert batch_size is not None, "eval batchSize must not be None"
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(rearrange)
    #################################
    
    # 평가이므로 1회만 동작 시킨다.
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()



eval_result = classifier.evaluate(input_fn=lambda: eval_input_fn(
    eval_input_enc, eval_output_dec, eval_target_dec,  DEFINES.batch_size))

print('\nEVAL set accuracy: {accuracy:0.3f}\n'.format(**eval_result))



INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-02T13:13:09Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_outseq2/check_point\model.ckpt-24676
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-03-02-13:13:51
INFO:tensorflow:Saving dict for global step 24676: accuracy = 0.81581753, global_step = 24676, loss = 2.1726224
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 24676: ./data_outseq2/check_point\model.ckpt-24676

EVAL set accuracy: 0.816



    INFO:tensorflow:Calling model_fn.
    INFO:tensorflow:Done calling model_fn.
    INFO:tensorflow:Starting evaluation at 2020-03-02T13:13:09Z
    INFO:tensorflow:Graph was finalized.
    INFO:tensorflow:Restoring parameters from ./data_outseq2/check_point\model.ckpt-24676
    INFO:tensorflow:Running local_init_op.
    INFO:tensorflow:Done running local_init_op.
    INFO:tensorflow:Finished evaluation at 2020-03-02-13:13:51
    INFO:tensorflow:Saving dict for global step 24676: accuracy = 0.81581753, global_step = 24676, loss = 2.1726224
    INFO:tensorflow:Saving 'checkpoint_path' summary for global step 24676: ./data_outseq2/check_point\model.ckpt-24676

    EVAL set accuracy: 0.816

# 테스트 하기

In [16]:
# 인코딩 데이터를 만드는 함수이며 
# 인덱스화 할 value와 키가 단어이고 값이 인덱스인 딕셔너리를 받아
# 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.  


def enc_processing(value, dictionary):
    sequences_input_index = []
    sequences_length = []
    # 형태소 토크나이징 사용 유무
    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)

    for sequence in value:
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        sequence_index = []
        # 문장을 스페이스 단위로 자르고 있다.
        for word in sequence.split():
            # 잘려진 단어들이 딕셔너리에 존재 하는지 보고 
            # 그 값을 가져와 sequence_index에 추가한다.
            if dictionary.get(word) is not None:
                sequence_index.extend([dictionary[word]])
            # 잘려진 단어가 딕셔너리에 존재 하지 않는 
            # 경우 이므로 UNK(2)를 넣어 준다.
            else:
                sequence_index.extend([dictionary[UNK]])
        
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length]

        sequences_length.append(len(sequence_index))
        # max_sequence_length보다 문장 길이가 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        sequences_input_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다. 
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 사전 작업이다.
    return np.asarray(sequences_input_index), sequences_length






# 디코딩 입력 데이터를 만드는 함수이다.
def dec_input_processing(value, dictionary):
    sequences_output_index = []
    sequences_length = []

    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)

    for sequence in value:
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        sequence_index = []
        # 디코딩 입력의 처음에는 START가 와야 하므로 
        # 그 값을 넣어 주고 시작한다.
        sequence_index = [dictionary[STD]] + [dictionary[word] for word in sequence.split()]

        if len(sequence_index) > DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length]
        sequences_length.append(len(sequence_index))
        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        sequences_output_index.append(sequence_index)

    return np.asarray(sequences_output_index), sequences_length



# 디코딩 출력 데이터를 만드는 함수이다.
def dec_target_processing(value, dictionary):
    sequences_target_index = []

    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)
    for sequence in value:
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 문장에서 스페이스 단위별로 단어를 가져와서 
        # 딕셔너리의 값인 인덱스를 넣어 준다.
        # 디코딩 출력의 마지막에 END를 넣어 준다.
        sequence_index = [dictionary[word] for word in sequence.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        # 그리고 END 토큰을 넣어 준다
        if len(sequence_index) >= DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length-1] + [dictionary[END]]
        else:
            sequence_index += [dictionary[END]]
        # max_sequence_length보다 문장 길이가 

        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        sequences_target_index.append(sequence_index)

    return np.asarray(sequences_target_index)


######################################################################################


# 인덱스를 스트링으로 변경하는 함수이다.
def pred2string(value, dictionary):
    sentence_string = []
    for v in value:
        # 딕셔너리에 있는 단어로 변경해서 배열에 담는다.
        sentence_string = [dictionary[index] for index in v['indexs']]

    print(sentence_string)
    answer = ""
    # 패딩값과 엔드값이 담겨 있으므로 패딩은 모두 스페이스 처리 한다.
    for word in sentence_string:
        if word not in PAD and word not in END:
            answer += word
            answer += " "

    print(answer)
    return answer


# 챗봇하기 (input - output)

In [17]:


input = "가끔 궁금해"

#######################

# 테스트용 데이터 만드는 부분이다.
# 인코딩 부분 만든다.
predic_input_enc, predic_input_enc_length = enc_processing([input], char2idx)


# 학습 과정이 아니므로 디코딩 입력은 
# 존재하지 않는다.(구조를 맞추기 위해 넣는다.)

predic_output_dec, predic_output_decLength = dec_input_processing([""], char2idx)       
# 학습 과정이 아니므로 디코딩 출력 부분도 
# 존재하지 않는다.(구조를 맞추기 위해 넣는다.)


predic_target_dec = dec_target_processing([""], char2idx)      

# 예측을 하는 부분이다.


predictions = classifier.predict(
    input_fn=lambda:  eval_input_fn(predic_input_enc, predic_output_dec, predic_target_dec, DEFINES.batch_size))

# 예측한 값을 인지 할 수 있도록 
# 텍스트로 변경하는 부분이다.


pred2string(predictions, idx2char)
##################################




100%|███████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.23it/s]

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_outseq2/check_point\model.ckpt-24676
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
['사랑', '사람', '사람', '에', '의', '의', '에', '에', '에', '에', '의', '과', '과', '과', '생각', '생각', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>']
사랑 사람 사람 에 의 의 에 에 에 에 의 과 과 과 생각 생각 


'사랑 사람 사람 에 의 의 에 에 에 에 의 과 과 과 생각 생각 '

    INFO:tensorflow:Calling model_fn.

    INFO:tensorflow:Done calling model_fn.
    INFO:tensorflow:Graph was finalized.
    INFO:tensorflow:Restoring parameters from ./data_outseq2/check_point\model.ckpt-24676
    INFO:tensorflow:Running local_init_op.
    INFO:tensorflow:Done running local_init_op.
    ['사랑', '사람', '사람', '에', '의', '의', '에', '에', '에', '에', '의', '과', '과', '과', '생각', '생각', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>']
    사랑 사람 사람 에 의 의 에 에 에 에 의 과 과 과 생각 생각 

# 6.4 :   Transformer 모델 챗봇